In [2]:
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
df_clean = pd.read_csv("outputs/CBECS_2012.csv")
df_clean.shape

(6720, 1119)

In [10]:
# Normalise the energy consumption (for degree days and area variation)
df_clean['EUI'] = df_clean['MFBTU']/df_clean['SQFT']
df_clean['DD'] = df_clean['HDD65']+df_clean['CDD65']
df_clean['NEUI'] = df_clean['EUI']/df_clean['DD']

In [11]:
df_clean[['HDD65','CDD65','DD','EUI','NEUI','MFBTU','ELBTU']].describe()

,HDD65,CDD65,DD,EUI,NEUI,MFBTU,ELBTU
count,6720.000000,6720.000000,6720.000000,6554.000000,6.554000e+03,6.554000e+03,6.554000e+03
mean,3568.630804,1661.645982,5230.276786,101.599890,2.097996e-02,1.746590e+07,9.283681e+06
std,1985.348720,1033.602459,1380.837904,122.620742,2.835682e-02,6.130714e+07,3.217463e+07
min,47.000000,12.000000,1652.000000,0.001429,2.545112e-07,3.000000e+00,0.000000e+00
25%,1997.500000,944.000000,4345.750000,33.479153,6.571500e-03,2.681425e+05,1.576400e+05
50%,3538.000000,1426.500000,5241.500000,64.397452,1.243813e-02,1.384251e+06,8.223465e+05
75%,5105.250000,2040.000000,6155.000000,120.568708,2.358406e-02,8.463776e+06,5.180308e+06
max,10744.000000,5979.000000,10760.000000,1340.194000,4.423495e-01,1.481866e+09,1.360596e+09


In [8]:
df_clean["EUI"] = df_clean["EUI"]*1000
df_clean["EUI"].describe()

count    6.554000e+03
mean     1.015999e+05
std      1.226207e+05
min      1.428571e+00
25%      3.347915e+04
50%      6.439745e+04
75%      1.205687e+05
max      1.340194e+06
Name: EUI, dtype: float64

### <font color='red'>Clean</font> #

In [2]:
df_clean = pd.read_csv("outputs/df_clean.csv")
df_clean.shape

(6662, 1516)

In [3]:
df_clean.head()

,SQFT,BASEMNT,ACT1PCT,ACT2PCT,ACT3PCT,EDSEAT,FDSEAT,OCCUPYP,LODOCCP,WKHRS,...,XRAYN,XRAYN_bin.2,RFGCOMPN,RFGCOMPN_bin.2,SERVERN_bin..1,SERVERN,SERVERN_bin.2,TVVIDEON_bin..1,TVVIDEON,TVVIDEON_bin.2
0,2400.0,0,100,0,0,0,0,0,0,16,...,0,0,0,0,0,0,0,0,0,0
1,114000.0,0,100,0,0,0,0,42,0,72,...,0,0,10,0,0,7,0,0,49,0
2,2550.0,0,100,0,0,0,0,0,0,40,...,0,0,0,0,0,0,0,0,0,0
3,500000.0,1,100,0,0,0,0,0,56,168,...,0,0,8,0,0,4,0,0,0,1
4,30000.0,0,50,50,0,0,0,100,0,65,...,0,0,2,0,0,10,0,0,1,0


### <font color='red'>Create NEUI</font> #

In [4]:
# Normalise the energy consumption (for degree days and area variation)
df_clean['EUI'] = df_clean['MFBTU']/df_clean['SQFT']
df_clean['DD'] = df_clean['HDD65']+df_clean['CDD65']
df_clean['NEUI'] = df_clean['EUI']/df_clean['DD']

In [5]:
df_clean[['HDD65','CDD65','DD','EUI','NEUI','MFBTU','ELBTU']].head()

,HDD65,CDD65,DD,EUI,NEUI,MFBTU,ELBTU
0,6647,1100,7747,0.011222,0.000001,26.934,20.748
1,3659,1634,5293,0.051664,0.000010,5889.672,5397.774
2,5079,899,5978,0.015252,0.000003,38.893,38.893
3,2399,2290,4689,0.096874,0.000021,48437.214,37301.178
4,813,4144,4957,0.063895,0.000013,1916.844,1526.710


In [6]:
#a = df_clean.isnull().sum()
#a.to_csv('a.csv')

In [7]:
# remove EUI
df_clean.drop('EUI', axis=1, inplace=True) 

#remove nans
df_clean.drop('NGBTU', axis=1, inplace=True) 
df_clean.drop('FKBTU', axis=1, inplace=True)
df_clean.drop('DHBTU', axis=1, inplace=True)
df_clean.drop('MFBTU', axis=1, inplace=True) 
df_clean.drop('ELBTU', axis=1, inplace=True) 

df_clean.shape

(6662, 1513)

In [8]:
# remove nan rows
df_clean = df_clean.apply (pd.to_numeric, errors='coerce')
df_clean = df_clean.dropna()
df_clean.shape

(6501, 1513)

In [9]:
# Chekc if any nans left
df_clean.isnull().values.any()

False

In [10]:
# Remove outliers

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
    
for col in df_clean.columns:
    replace_with_thresholds(df_clean, col)

In [11]:
#df_clean = df_clean.astype(float)
df_clean[df_clean < 0] = 0

TRIM DF

In [12]:
trim_df = df_clean
trim_df.to_csv('outputs/trim_df.csv')
trim_df.shape

(6501, 1513)

In [13]:
trim_df.head()

,SQFT,BASEMNT,ACT1PCT,ACT2PCT,ACT3PCT,EDSEAT,FDSEAT,OCCUPYP,LODOCCP,WKHRS,...,RFGCOMPN,RFGCOMPN_bin.2,SERVERN_bin..1,SERVERN,SERVERN_bin.2,TVVIDEON_bin..1,TVVIDEON,TVVIDEON_bin.2,DD,NEUI
0,2400.0,0.0,100,0,0,0,0,0,0,16,...,0.0,0,0,0,0,0,0.0,0.0,7747,0.000001
1,114000.0,0.0,100,0,0,0,0,42,0,72,...,10.0,0,0,7,0,0,49.0,0.0,5293,0.000010
2,2550.0,0.0,100,0,0,0,0,0,0,40,...,0.0,0,0,0,0,0,0.0,0.0,5978,0.000003
3,500000.0,1.0,100,0,0,0,0,0,56,168,...,8.0,0,0,4,0,0,0.0,1.0,4689,0.000021
4,30000.0,0.0,50,50,0,0,0,100,0,65,...,2.0,0,0,10,0,0,1.0,0.0,4957,0.000013
